In [4]:
import os
import random
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, random_split
import torch.nn as nn
import torch.optim as optim
import torch
import time
import copy
from PIL import Image

# Define the dataset directory
data_dir = 'C:/Users/naiks/OneDrive/Documents/CodeClause/Crop Disease Identification/Dataset/PlantVillage'

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the dataset
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Use only 10% of the dataset
num_samples = int(1 * len(full_dataset))
sampled_indices = random.sample(range(len(full_dataset)), num_samples)
sampled_dataset = Subset(full_dataset, sampled_indices)

# Split the sampled dataset into training and validation sets (80% train, 20% val)
train_size = int(0.8 * len(sampled_dataset))
val_size = len(sampled_dataset) - train_size
train_dataset, val_dataset = random_split(sampled_dataset, [train_size, val_size])

# Apply the appropriate transformations to the validation set
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

# Create dataloaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes

# Verify the dataset
print(f"Classes: {class_names}")
print(f"Number of training images: {dataset_sizes['train']}")
print(f"Number of validation images: {dataset_sizes['val']}")

Classes: ['Pepper Bell - Bacterial spot', 'Pepper Bell - Healthy', 'Potato - Early Blight', 'Potato - Late Blight', 'Potato - healthy', 'Tomato - Bacterial Spot', 'Tomato - Early Blight', 'Tomato - Healthy', 'Tomato - Late Blight', 'Tomato - Leaf Mold', 'Tomato - Mosaic Virus', 'Tomato - Septoria Leaf Spot', 'Tomato - Spider Mites', 'Tomato - Target Spot', 'Tomato - YellowLeaf Curl Virus']
Number of training images: 16510
Number of validation images: 4128


In [5]:
# Load pre-trained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)

# Modify the final layer
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(class_names))

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_model(model, criterion, optimizer, num_epochs=20):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=10)

# Save the model
torch.save(model.state_dict(), 'plantvillage_mobilenetv2.pth')

c:\Users\naiks\anaconda3\envs\crop\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\naiks\anaconda3\envs\crop\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/9
----------
train Loss: 0.5480 Acc: 0.8329
val Loss: 0.1960 Acc: 0.9351

Epoch 1/9
----------
train Loss: 0.2431 Acc: 0.9217
val Loss: 0.1177 Acc: 0.9641

Epoch 2/9
----------
train Loss: 0.1898 Acc: 0.9393
val Loss: 0.0924 Acc: 0.9712

Epoch 3/9
----------
train Loss: 0.1696 Acc: 0.9443
val Loss: 0.0809 Acc: 0.9777

Epoch 4/9
----------
train Loss: 0.1431 Acc: 0.9524
val Loss: 0.0979 Acc: 0.9702

Epoch 5/9
----------
train Loss: 0.1326 Acc: 0.9564
val Loss: 0.0616 Acc: 0.9818

Epoch 6/9
----------
train Loss: 0.1175 Acc: 0.9606
val Loss: 0.0840 Acc: 0.9736

Epoch 7/9
----------
train Loss: 0.1075 Acc: 0.9649
val Loss: 0.0609 Acc: 0.9809

Epoch 8/9
----------
train Loss: 0.1089 Acc: 0.9631
val Loss: 0.0661 Acc: 0.9809

Epoch 9/9
----------
train Loss: 0.0968 Acc: 0.9694
val Loss: 0.0641 Acc: 0.9816

Training complete in 10m 22s
Best val Acc: 0.9818


In [7]:
# Define the image transformation for the input image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predict(image_path):
    image = Image.open(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        predicted_class = class_names[preds[0]]
        return predicted_class

# Predict on a new image
image_path = 'C:/Users/naiks/OneDrive/Documents/CodeClause/Crop Disease Identification/Dataset/PlantVillage/Pepper Bell - Bacterial spot/0b47ce18-7cfe-45e8-b21e-b83cb6282455___JR_B.Spot 3162.JPG'
predicted_class = predict(image_path)
print(f'Predicted class: {predicted_class}')

Predicted class: Pepper Bell - Bacterial spot
